# 04. LSTM Windowing for Time-Series Input

이 노트북에서는 LSTM 같은 시계열 딥러닝 모델에 사용할 수 있도록
**슬라이딩 윈도우 방식**으로 데이터를 전처리합니다.
- 입력: `(window_size, feature_dim)` 형태의 시계열 텐서
- 타깃: 해당 시퀀스 마지막 시점의 RUL


In [1]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import json

# 전처리된 train 데이터 로드
df_selected = pd.read_csv('../data/processed_train.csv')


In [2]:
print(df_selected.columns)

Index(['unit', 'time', 'op1', 'op2', 's2', 's3', 's4', 's5', 's6', 's7', 's8',
       's9', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's20', 's21',
       'RUL', 'RUL_clipped'],
      dtype='object')


In [3]:
# LSTM 학습용 슬라이딩 윈도우 생성 함수
def make_lstm_dataset(df, features, target, window_size=30):
    X, y = [], []
    for unit in df['unit'].unique():
        unit_df = df[df['unit'] == unit].reset_index(drop=True)
        for i in range(len(unit_df) - window_size + 1):
            window = unit_df.iloc[i:i+window_size][features].values
            label = unit_df.iloc[i+window_size-1][target]
            X.append(window)
            y.append(label)
    return np.array(X), np.array(y)

In [4]:
# features, target 정의
with open('../eda/selected_sensors.json') as f:
    selected_sensors = json.load(f)
features = ['op1', 'op2'] + selected_sensors
target = 'RUL_clipped'

In [5]:
# 윈도우 적용
X_lstm, y_lstm = make_lstm_dataset(df_selected, features, target, window_size=30)
print("X shape:", X_lstm.shape)  # (샘플 수, window_size, feature 수)
print("y shape:", y_lstm.shape)

X shape: (17731, 30, 19)
y shape: (17731,)


In [6]:
np.save('../models/X_lstm.npy', X_lstm)
np.save('../models/y_lstm.npy', y_lstm)

In [7]:
print("X shape:", X_lstm.shape)
print("y shape:", y_lstm.shape)

X shape: (17731, 30, 19)
y shape: (17731,)
